# 6.2.2 圖像拼接專案 - 中級練習

本練習專注於圖像拼接技術的實際應用，通過創建全景圖像來掌握特徵匹配、幾何變換和圖像融合技術。

## 練習目標
- 掌握圖像拼接的完整流程
- 實現自動特徵匹配和配準
- 學習圖像融合和接縫消除技術
- 處理光照和曝光差異
- 優化拼接品質和性能

## 難度等級: ⭐⭐⭐ (中級)

In [ ]:
import cv2
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
import time
from typing import List, Tuple, Optional

sys.path.append('../../utils')
from image_utils import load_image, resize_image
from visualization import display_image, display_multiple_images
from performance import time_function

plt.rcParams['font.sans-serif'] = ['DejaVu Sans']
plt.rcParams['axes.unicode_minus'] = False

print("✅ 環境設置完成")

## 挑戰1: 實現基礎圖像拼接器

In [ ]:
class ImageStitcher:
    """圖像拼接器類"""
    
    def __init__(self):
        """初始化拼接器"""
        self.detector = cv2.SIFT_create()
        
        # 創建FLANN匹配器
        FLANN_INDEX_KDTREE = 1
        index_params = dict(algorithm=FLANN_INDEX_KDTREE, trees=5)
        search_params = dict(checks=50)
        self.matcher = cv2.FlannBasedMatcher(index_params, search_params)
        
        print("✅ 圖像拼接器初始化完成")
    
    def extract_features(self, image):
        """提取圖像特徵"""
        if len(image.shape) == 3:
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        else:
            gray = image
            
        kp, desc = self.detector.detectAndCompute(gray, None)
        return kp, desc
    
    def match_features(self, desc1, desc2, ratio_threshold=0.75):
        """匹配特徵點"""
        matches = self.matcher.knnMatch(desc1, desc2, k=2)
        
        good_matches = []
        for match_pair in matches:
            if len(match_pair) == 2:
                m, n = match_pair
                if m.distance < ratio_threshold * n.distance:
                    good_matches.append(m)
        
        return good_matches
    
    def find_homography(self, kp1, kp2, matches):
        """計算單應性矩陣"""
        if len(matches) < 4:
            return None, None
        
        src_pts = np.float32([kp1[m.queryIdx].pt for m in matches]).reshape(-1, 1, 2)
        dst_pts = np.float32([kp2[m.trainIdx].pt for m in matches]).reshape(-1, 1, 2)
        
        H, mask = cv2.findHomography(src_pts, dst_pts, 
                                    cv2.RANSAC, 5.0)
        
        return H, mask
    
    def stitch_two_images(self, img1, img2):
        """拼接兩張圖像"""
        print("🔄 開始拼接兩張圖像...")
        
        # 提取特徵
        kp1, desc1 = self.extract_features(img1)
        kp2, desc2 = self.extract_features(img2)
        
        print(f"  圖像1特徵點: {len(kp1)}")
        print(f"  圖像2特徵點: {len(kp2)}")
        
        if desc1 is None or desc2 is None:
            print("❌ 無法提取特徵")
            return None
        
        # 匹配特徵
        matches = self.match_features(desc1, desc2)
        print(f"  匹配特徵點: {len(matches)}")
        
        if len(matches) < 10:
            print("❌ 匹配點不足")
            return None
        
        # 計算單應性矩陣
        H, mask = self.find_homography(kp1, kp2, matches)
        
        if H is None:
            print("❌ 無法計算單應性矩陣")
            return None
        
        inliers = np.sum(mask) if mask is not None else 0
        print(f"  內點數量: {inliers}/{len(matches)}")
        
        # 計算拼接畫布大小
        h1, w1 = img1.shape[:2]
        h2, w2 = img2.shape[:2]
        
        # 變換img1的角點到img2的座標系
        corners1 = np.float32([[0, 0], [w1, 0], [w1, h1], [0, h1]]).reshape(-1, 1, 2)
        transformed_corners = cv2.perspectiveTransform(corners1, H)
        
        # 合併所有角點
        all_corners = np.concatenate([transformed_corners, 
                                     np.float32([[0, 0], [w2, 0], [w2, h2], [0, h2]]).reshape(-1, 1, 2)])
        
        # 計算邊界
        x_min, y_min = np.int32(all_corners.min(axis=0).ravel())
        x_max, y_max = np.int32(all_corners.max(axis=0).ravel())
        
        # 調整變換矩陣以處理負座標
        translation = np.array([[1, 0, -x_min], [0, 1, -y_min], [0, 0, 1]])
        H_adjusted = translation @ H
        
        # 計算輸出尺寸
        output_width = x_max - x_min
        output_height = y_max - y_min
        
        print(f"  輸出尺寸: {output_width}x{output_height}")
        
        # 創建拼接結果
        stitched = cv2.warpPerspective(img1, H_adjusted, (output_width, output_height))
        
        # 將第二張圖像放置在正確位置
        stitched[-y_min:-y_min+h2, -x_min:-x_min+w2] = img2
        
        print("✅ 基礎拼接完成")
        return stitched

# 創建拼接器
stitcher = ImageStitcher()

## 挑戰2: 高級圖像融合

In [ ]:
class AdvancedImageStitcher(ImageStitcher):
    """進階圖像拼接器，包含高級融合技術"""
    
    def __init__(self):
        super().__init__()
        
    def blend_images(self, img1, img2, mask):
        """圖像融合處理"""
        # 多頻帶融合
        return self.multiband_blending(img1, img2, mask)
    
    def multiband_blending(self, img1, img2, mask, levels=4):
        """多頻帶融合算法"""
        # 建立高斯金字塔
        GP1 = [img1.copy()]
        GP2 = [img2.copy()]
        
        for i in range(levels):
            GP1.append(cv2.pyrDown(GP1[i]))
            GP2.append(cv2.pyrDown(GP2[i]))
        
        # 建立拉普拉斯金字塔
        LP1 = [GP1[levels]]
        LP2 = [GP2[levels]]
        
        for i in range(levels, 0, -1):
            size = (GP1[i-1].shape[1], GP1[i-1].shape[0])
            L1 = cv2.subtract(GP1[i-1], cv2.pyrUp(GP1[i], dstsize=size))
            L2 = cv2.subtract(GP2[i-1], cv2.pyrUp(GP2[i], dstsize=size))
            LP1.append(L1)
            LP2.append(L2)
        
        # 融合拉普拉斯金字塔
        LS = []
        for l1, l2 in zip(LP1, LP2):
            ls = l1 * 0.5 + l2 * 0.5  # 簡化融合
            LS.append(ls)
        
        # 重建圖像
        result = LS[0]
        for i in range(1, len(LS)):
            size = (LS[i].shape[1], LS[i].shape[0])
            result = cv2.add(cv2.pyrUp(result, dstsize=size), LS[i])
        
        return result
    
    def stitch_multiple_images(self, images):
        """拼接多張圖像"""
        if len(images) < 2:
            return images[0] if images else None
        
        print(f"🔄 開始拼接 {len(images)} 張圖像...")
        
        result = images[0]
        
        for i in range(1, len(images)):
            print(f"  拼接第 {i+1} 張圖像...")
            
            stitched = self.stitch_two_images(result, images[i])
            
            if stitched is not None:
                result = stitched
                print(f"  ✅ 第 {i+1} 張圖像拼接成功")
            else:
                print(f"  ❌ 第 {i+1} 張圖像拼接失敗")
                break
        
        return result

# 創建進階拼接器
advanced_stitcher = AdvancedImageStitcher()
print("✅ 進階圖像拼接器初始化完成")

## 練習任務: 創建全景圖像

In [ ]:
def create_test_images():
    """創建測試圖像序列"""
    # 創建模擬的重疊圖像序列
    base_image = np.random.randint(100, 200, (300, 400, 3), dtype=np.uint8)
    
    images = []
    
    for i in range(3):
        # 創建重疊區域
        img = base_image.copy()
        
        # 添加獨特特徵
        cv2.rectangle(img, (50 + i*80, 50), (150 + i*80, 150), (255, 0, 0), -1)
        cv2.circle(img, (200 + i*60, 200), 30, (0, 255, 0), -1)
        cv2.putText(img, f"Image {i+1}", (100 + i*70, 250), 
                   cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
        
        images.append(img)
    
    return images

# 創建測試圖像
test_images = create_test_images()

print(f"📸 創建了 {len(test_images)} 張測試圖像")

# 顯示測試圖像
display_multiple_images(test_images, 
                       [f"圖像 {i+1}" for i in range(len(test_images))],
                       figsize=(15, 5))

In [ ]:
# 執行圖像拼接
print("🎬 開始創建全景圖像...")

# 使用OpenCV內建的Stitcher (比較基準)
try:
    opencv_stitcher = cv2.Stitcher.create()
    status, opencv_panorama = opencv_stitcher.stitch(test_images)
    
    if status == cv2.Stitcher_OK:
        print("✅ OpenCV Stitcher成功")
    else:
        print(f"❌ OpenCV Stitcher失敗: {status}")
        opencv_panorama = None
        
except Exception as e:
    print(f"❌ OpenCV Stitcher不可用: {e}")
    opencv_panorama = None

# 使用自定義拼接器
start_time = time.time()
custom_panorama = advanced_stitcher.stitch_multiple_images(test_images)
custom_time = (time.time() - start_time) * 1000

if custom_panorama is not None:
    print(f"✅ 自定義拼接器成功，耗時: {custom_time:.1f}ms")
    
    # 顯示結果比較
    results_to_show = []
    titles = []
    
    # 顯示原始圖像組合
    combined_original = np.hstack(test_images)
    results_to_show.append(combined_original)
    titles.append("原始圖像序列")
    
    # 顯示自定義結果
    results_to_show.append(custom_panorama)
    titles.append(f"自定義拼接結果\n{custom_time:.1f}ms")
    
    # 顯示OpenCV結果（如果可用）
    if opencv_panorama is not None:
        results_to_show.append(opencv_panorama)
        titles.append("OpenCV Stitcher結果")
    
    display_multiple_images(results_to_show, titles, figsize=(15, 10))
    
else:
    print("❌ 自定義拼接器失敗")

## 挑戰3: 品質評估和優化

In [ ]:
def evaluate_stitching_quality(panorama, original_images):
    """評估拼接品質"""
    metrics = {}
    
    # 銳度評估
    gray = cv2.cvtColor(panorama, cv2.COLOR_BGR2GRAY)
    laplacian = cv2.Laplacian(gray, cv2.CV_64F)
    metrics['sharpness'] = laplacian.var()
    
    # 對比度評估
    metrics['contrast'] = np.std(gray)
    
    # 接縫檢測
    edges = cv2.Canny(gray, 50, 150)
    seam_pixels = np.sum(edges > 0)
    metrics['seam_visibility'] = seam_pixels / (panorama.shape[0] * panorama.shape[1])
    
    return metrics

# 評估拼接品質
if custom_panorama is not None:
    quality_metrics = evaluate_stitching_quality(custom_panorama, test_images)
    
    print("📊 拼接品質評估:")
    print(f"  銳度: {quality_metrics['sharpness']:.1f}")
    print(f"  對比度: {quality_metrics['contrast']:.1f}")
    print(f"  接縫可見度: {quality_metrics['seam_visibility']:.4f}")
    
    # 品質評級
    if quality_metrics['sharpness'] > 100 and quality_metrics['seam_visibility'] < 0.1:
        grade = "優秀"
    elif quality_metrics['sharpness'] > 50:
        grade = "良好"
    else:
        grade = "需改善"
    
    print(f"🏆 拼接品質: {grade}")

## 總結與延伸

### 完成技能檢核
- [ ] 實現了基礎的兩圖像拼接
- [ ] 完成了多圖像序列拼接
- [ ] 理解了特徵匹配和幾何變換
- [ ] 實現了品質評估機制
- [ ] 能處理拼接失敗的情況

### 進階挑戦方向
- 實現更好的圖像融合算法
- 添加曝光補償功能
- 處理魚眼鏡頭畸變
- 實現柱面投影拼接
- 優化大圖像的處理性能